<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<b>Little Demo: 3D Plots with the Plotly Python Package</b> <br>
Contact author(s): Melissa Graham <br>
Last verified to run: 2024-07-30 <br>
LSST Science Pipelines version: Weekly 2024_16 <br>
Container Size: medium

This little demo has used the examples of 3D charts from [plotly.com/python](https://plotly.com/python/)

>**Warning:** This notebook does not respond well to using "Restart kernel and run all cells". Better to execute one cell at a time.

## Set up

In [ ]:
import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
import healsparse as hsp

from lsst.daf.butler import Butler
from lsst.rsp import get_tap_service

## Plotly-provided demos

### 3D scatter plot

The exact demo from [plotly.com/python/3d-scatter-plots/](https://plotly.com/python/3d-scatter-plots/).

In [ ]:
df = px.data.iris()

Note that `df` is a pandas dataframe.

In [ ]:
# df

In [ ]:
# help(df)

Use the `scatter_3d` method.

In [ ]:
# help(px.scatter_3d)

In [ ]:
fig = px.scatter_3d(df, x='sepal_length', y='sepal_width', z='petal_width',
              color='species')
fig.show()

### Surface plots

From [plotly.com/python/3d-surface-plots/](https://plotly.com/python/3d-surface-plots/).

In [ ]:
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
fig = go.Figure(data=[go.Surface(z=z_data.values)])
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
z_data = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/api_docs/mt_bruno_elevation.csv')
z = z_data.values
sh_0, sh_1 = z.shape
x, y = np.linspace(0, 1, sh_0), np.linspace(0, 1, sh_1)
fig = go.Figure(data=[go.Surface(z=z, x=x, y=y)])
fig.update_layout(title='Mt Bruno Elevation', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

In [ ]:
# x

In [ ]:
# y

In [ ]:
# z

In [ ]:
del z_data, z, x, y, sh_0, sh_1

## DP0.2 Object catalog demos

Start TAP service.

In [ ]:
service = get_tap_service("tap")

Retrieve 10000 `Object`s with $griz$ apparent magnitudes brighter than 25th mag.

In [ ]:
query = """SELECT TOP 10000 
        coord_ra, coord_dec, 
        scisql_nanojanskyToAbMag(g_cModelFlux) as gmag, 
        scisql_nanojanskyToAbMag(r_cModelFlux) as rmag, 
        scisql_nanojanskyToAbMag(i_cModelFlux) as imag, 
        scisql_nanojanskyToAbMag(z_cModelFlux) as zmag, 
        refExtendedness, footprintArea 
        FROM dp02_dc2_catalogs.Object 
        WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), 
        CIRCLE('ICRS', 62, -37, 1.0)) = 1 
        AND detect_isPrimary = 1 
        AND g_cModelFlux > 360 
        AND r_cModelFlux > 360 
        AND i_cModelFlux > 360 
        AND z_cModelFlux > 360"""
print(query)

In [ ]:
job = service.submit_job(query)
job.run()
job.wait(phases=['COMPLETED', 'ERROR'])
print('Job phase is', job.phase)

In [ ]:
# job.raise_if_error()

In [ ]:
results = job.fetch_result().to_table().to_pandas()

In [ ]:
results['grc'] = results['gmag'] - results['rmag']
results['ric'] = results['rmag'] - results['imag']
results['izc'] = results['imag'] - results['zmag']

Investigate the data values.

Recall that:
 * footprintArea is the number of pixels associated with the `Object`
 * extendedness is 0 is no, not extended (point-like); 1 is yes, extended (not point-like)
 * all point-like objects will also have the smallest `footprintArea`

In [ ]:
print('gmag ', np.min(results['gmag']), np.max(results['gmag']))
print('rmag ', np.min(results['rmag']), np.max(results['rmag']))
print('imag ', np.min(results['imag']), np.max(results['imag']))
print('zmag ', np.min(results['zmag']), np.max(results['zmag']))
print('grc  ', np.min(results['grc']), np.max(results['grc']))
print('ric  ', np.min(results['ric']), np.max(results['ric']))
print('izc  ', np.min(results['izc']), np.max(results['izc']))
print('area ', np.min(results['footprintArea']), np.max(results['footprintArea']))

tx0 = np.where(results['refExtendedness'] == 0)[0]
tx1 = np.where(results['refExtendedness'] == 1)[0]
print('refExtendedness: N(0)='+str(len(tx0))+'  N(1)='+str(len(tx1)))
del tx0, tx1

### Magnitude-space

With points colored by $g-r$ color.

In [ ]:
fig = px.scatter_3d(results, x='gmag', y='rmag', z='imag',
                    color='grc')
fig.show()

### Color-space

With points colored by $i$-band magnitude, and the symbol representing `refExtendedness`.

In [ ]:
fig = px.scatter_3d(results, x='grc', y='ric', z='izc',
                    color='imag', symbol='refExtendedness', opacity=0.7)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
                                          ticks="outside",
                                          ticksuffix=" mag"))

fig.show()

As above, but make the size of the point related to footprintArea.

Since the point sources with extendedness of 0 all have the smallest footprint areas,
the fact that they're plotted with a diamond symbol is not really visible in the plot below.

In [ ]:
fig = px.scatter_3d(results, x='grc', y='ric', z='izc',
              color='imag', size='footprintArea', size_max=22,
              symbol='refExtendedness', opacity=0.7)

fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))

fig.update_layout(coloraxis_colorbar=dict(yanchor="top", y=1, x=0,
                                          ticks="outside",
                                          ticksuffix=" mag"))
fig.show()

In [ ]:
del service, query, job, results

## DP0.2 survey propert map demo

See the DP0.2 tutorial notebook on survey property maps.

Instantiate the butler.

In [ ]:
butler = Butler('dp02', collections='2.2i/runs/DP0.2')

Get the map for magnitude limits.

In [ ]:
hspmap = butler.get('deepCoadd_psf_maglim_consolidated_map_weighted_mean', band='i')

Extract 250 x 250 points from the map for a contrained region of sky.

In [ ]:
ra = np.linspace(59.5, 60.5, 250)
dec = np.linspace(-37.5, -36.5, 250)
x, y = np.meshgrid(ra, dec)
values = hspmap.get_values_pos(x, y)

Reformat for `go`.

In [ ]:
x_array = np.asarray(x[0][:], dtype='float')
tmpy = y.transpose()
y_array = np.asarray(tmpy[0][:])
del tmpy

To explore the format, uncomment any line and execute.

In [ ]:
# x
# y
# x_array
# y_array

Plot the $i$-band magnitude limit as a surface map.

In [ ]:
fig = go.Figure(data=[go.Surface(z=values, x=x_array, y=y_array)])

fig.update_layout(title='i-band magnitude limit map', autosize=False,
                  width=750, height=600)

fig.show()

In [ ]:
del butler, hspmap, ra, dec, x, y, values, x_array, y_array